In [3]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
import pandas as pd

In [4]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

finbert_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)

Device set to use cpu


In [5]:
def classify_financial_text(text):
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Convert logits to probabilities
    probs = F.softmax(outputs.logits, dim=-1)

    # Get the label mapping
    label_mapping = model.config.id2label  # {0: 'negative', 1: 'neutral', 2: 'positive'}

    # Convert to dictionary with labels
    result = {label_mapping[i]: float(probs[0][i]) for i in range(len(probs[0]))}

    return result

In [7]:
text_samples = [
    "Apple's stock price fell by 10% after disappointing earnings.",
    "The Federal Reserve maintained interest rates, causing market stability.",
    "Tesla's revenue hit record highs, boosting investor confidence."
]

for text in text_samples:
    print(f"Text: {text}")
    print("Sentiment Scores:", classify_financial_text(text))
    print()

Text: Apple's stock price fell by 10% after disappointing earnings.
Sentiment Scores: {'positive': 0.006525705102831125, 'negative': 0.972838819026947, 'neutral': 0.020635506138205528}

Text: The Federal Reserve maintained interest rates, causing market stability.
Sentiment Scores: {'positive': 0.34140244126319885, 'negative': 0.08414701372385025, 'neutral': 0.5744505524635315}

Text: Tesla's revenue hit record highs, boosting investor confidence.
Sentiment Scores: {'positive': 0.946237325668335, 'negative': 0.019426118582487106, 'neutral': 0.034336525946855545}



In [8]:
results = finbert_pipeline(text_samples)
for text, result in zip(text_samples, results):
    print(f"Text: {text}")
    print(f"Sentiment: {result['label']} (Confidence: {result['score']:.4f})")
    print()

Text: Apple's stock price fell by 10% after disappointing earnings.
Sentiment: negative (Confidence: 0.9728)

Text: The Federal Reserve maintained interest rates, causing market stability.
Sentiment: neutral (Confidence: 0.5745)

Text: Tesla's revenue hit record highs, boosting investor confidence.
Sentiment: positive (Confidence: 0.9462)

